In [18]:
# python3
# this script is about to read and write ionmix cn4 file type
# one can combine other eos data library like badger to run in FLASH
# based on flash center's opacplot

import numpy as np
import re
from io import StringIO
import matplotlib.pyplot as plt


In [19]:
f = open('al-imx-003.cn4','r')
global nt,nd,ng #numder of temperatures densities and energy groups
tempnums = 21
dennums = 25
atom = 73 #atomic number and fraction
frac = 1.0

#first several lines
nt = int(f.read(10))
nd = int(f.read(10))

for i in range(3):
    f.readline()

ng = 6
f.readline()
#get the strings and convert into numbers
txt = re.sub(r'\s','',f.read())




In [20]:
data = StringIO(txt)

#read the string by the moving of the pointer
def read_block(f,n):
    arr = np.zeros(n)
    for i in range(n):
        arr[i] = float(f.read(12))
    return arr

temps = read_block(data,nt)
dens = read_block(data,nd)
zbar = read_block(data,nd*nt).reshape(nd,nt)
dzdt = read_block(data,nd*nt).reshape(nd,nt)
pion = read_block(data,nd*nt).reshape(nd,nt)
pele = read_block(data,nd*nt).reshape(nd,nt)
dpidt = read_block(data,nd*nt).reshape(nd,nt)
dpedt = read_block(data,nd*nt).reshape(nd,nt)
eion = read_block(data,nd*nt).reshape(nd,nt)
eele = read_block(data,nd*nt).reshape(nd,nt)
deidt = read_block(data,nd*nt).reshape(nd,nt)
deedt = read_block(data,nd*nt).reshape(nd,nt)
deidn = read_block(data,nd*nt).reshape(nd,nt)
deedn = read_block(data,nd*nt).reshape(nd,nt)


In [21]:
opac_bounds = read_block(data,7)
arr_ro = read_block(data,nd*nt*ng)
arr_pa = read_block(data,nd*nt*ng)
arr_pe = read_block(data,ng*nd*nt)
rosseland = np.zeros((nd,nt,ng))
planck_absorb = np.zeros((nd,nt,ng))
planck_emiss = np.zeros((nd,nt,ng))

#be careful of the shape of the data
i = 0
for g in range(ng):
    for d in range(nd):
        for t in range(nt):
            rosseland[d,t,g] = arr_ro[i]
            planck_absorb[d,t,g] = arr_pa[i]
            planck_emiss[d,t,g] = arr_pe[i]
            i += 1


In [22]:
zbar_new = np.loadtxt('zbar.txt').reshape(nt,nd)
pion_new = np.loadtxt('i_pressure.txt').reshape(nt,nd)
pele_new = np.loadtxt('e_pressure.txt').reshape(nt,nd)
eion_new = np.loadtxt('i_energy.txt').reshape(nt,nd)
eele_new = np.loadtxt('e_energy.txt').reshape(nt,nd)

In [23]:
#fig,axes = plt.subplots(2,1)
#axes[0].imshow(zbar)
#axes[1].imshow(zbar_new.T)
#cbar = plt.colorbar()

In [24]:
np.shape(zbar) == np.shape(zbar_new)

False

In [25]:
#write ionmix format
fw = open('output.cn4','w')
fw.write('%10i%10i\n' % (nt,nd))
fw.write(" atomic #s of gases: ")
fw.write("%10i" % atom)
fw.write("\nrelative fractions: ")
fw.write("%10.2E" % frac)
fw.write('\n')
fw.write("%12i\n" % ng)

#convert the numbers back into string 
def convert(num):
    string_org = "%12.5E" % (num)
    negative = (string_org[0] == "-")
    lead = "-." if negative else "0."
    string = lead + string_org[1] + string_org[3:8] + "E"

    # Deal with the exponent:

    # Check for zero:
    if int(string_org[1] + string_org[3:8]) == 0:
        return string + "+00"

    # Not zero:
    expo = int(string_org[9:]) + 1
    if expo < 0:
        string += "-"
    else:
        string += "+"
    string += "%02d" % abs(expo)
    return string

def write_block(var):
    count = 0
    for n in range(len(var)):
        count += 1

        fw.write("%s" % convert(var[n]))

        if count == 4:
            count = 0
            fw.write("\n")

    if count != 0: fw.write("\n")

def write_opac_block(var):
    count = 0
    for g in range(ng):
        for jd in range(nd):
            for jt in range(nt):
                count += 1

                fw.write("%s" % convert(var[jd,jt,g]))

                if count == 4:
                    count = 0
                    fw.write("\n")

    if count != 0: fw.write("\n")
write_block(temps)
write_block(dens)
write_block(zbar_new.T.flatten()) #here the transpose to get the right position of the data
write_block(dzdt.flatten())
write_block(pion_new.T.flatten())
write_block(pele_new.T.flatten())
write_block(dpidt.flatten())
write_block(dpedt.flatten())
write_block(eion_new.T.flatten())
write_block(eele_new.T.flatten())
write_block(dpidt.flatten())
write_block(dpedt.flatten())
write_block(deidn.flatten())
write_block(deedn.flatten())
write_block(opac_bounds)
write_opac_block(rosseland)
write_opac_block(planck_absorb)
write_opac_block(planck_emiss)


